In [1]:
import pandas as pd
import pymongo
import selenium
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser

In [2]:
executable_path = {'executable_path': '../static/geckodriver'}
browser = Browser('firefox', **executable_path, headless=True)
# executable_path = {'executable_path': '../static/chromedriver.exe'}
# browser = Browser('chrome', **executable_path, headless=True)

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
event_df = pd.read_csv("../data/rolling_event_data.csv")
event_df

,Unnamed: 0,event,date,location,event_url
0,0,UFC Fight Night: Makhachev vs. Moises,"July 17, 2021","Las Vegas, Nevada, US",http://ufcstats.com/event-details/40389d39a92f...
1,0,UFC 264: Poirier vs. McGregor 3,"July 10, 2021","Las Vegas, Nevada, US",http://ufcstats.com/event-details/8a0be41c0380...
2,1,UFC Fight Night: Gane vs. Volkov,"June 26, 2021","Las Vegas, Nevada, US",http://ufcstats.com/event-details/a1d6b308781a...
3,2,UFC Fight Night: Jung vs. Ige,"June 19, 2021","Las Vegas, Nevada, US",http://ufcstats.com/event-details/9ddfb3369a3b...
4,3,UFC 263: Adesanya vs. Vettori 2,"June 12, 2021","Glendale, Arizona, US",http://ufcstats.com/event-details/0b64d0fed453...
...,...,...,...,...,...
564,563,UFC 6: Clash of the Titans,"July 14, 1995","Casper, Wyoming, US",http://ufcstats.com/event-details/1c3f5e85b59e...
565,564,UFC 5: The Return of the Beast,"April 07, 1995","Charlotte, North Carolina, US",http://ufcstats.com/event-details/dedc3bb440d0...
566,565,UFC 4: Revenge of the Warriors,"December 16, 1994","Tulsa, Oklahoma, US",http://ufcstats.com/event-details/b60391da771d...
567,566,UFC 3: The American Dream,"September 09, 1994","Charlotte, North Carolina, US",http://ufcstats.com/event-details/1a49e0670dfa...


In [5]:
event_urls = event_df["event_url"].tolist()

In [6]:
fight_urls_df = pd.read_csv("../data/rolling_fight_urls.csv")
fight_urls_df

,Unnamed: 0,fight_url
0,0,http://ufcstats.com/fight-details/7d0ce28ce8a7...
1,1,http://ufcstats.com/fight-details/8245791a3d34...
2,2,http://ufcstats.com/fight-details/53b480b44b0b...
3,3,http://ufcstats.com/fight-details/3740c8033e31...
4,4,http://ufcstats.com/fight-details/dbcc5b5d568e...
...,...,...
6161,6161,http://ufcstats.com/fight-details/3b020d4914b4...
6162,6162,http://ufcstats.com/fight-details/d917c8c7461b...
6163,6163,http://ufcstats.com/fight-details/ccee020be2e8...
6164,6164,http://ufcstats.com/fight-details/4b9ae533ccb3...


In [7]:
fight_urls = fight_urls_df["fight_url"].tolist()
fight_urls

['http://ufcstats.com/fight-details/7d0ce28ce8a77eb1',
 'http://ufcstats.com/fight-details/8245791a3d3403c6',
 'http://ufcstats.com/fight-details/53b480b44b0b51f1',
 'http://ufcstats.com/fight-details/3740c8033e31c90d',
 'http://ufcstats.com/fight-details/dbcc5b5d568efa8b',
 'http://ufcstats.com/fight-details/7070abfa26c079b8',
 'http://ufcstats.com/fight-details/96bbb6c573dac27a',
 'http://ufcstats.com/fight-details/07468b6347ac5e3d',
 'http://ufcstats.com/fight-details/858feabaf429a963',
 'http://ufcstats.com/fight-details/5d5bf07dbfada0e3',
 'http://ufcstats.com/fight-details/1aba46988c9b58f9',
 'http://ufcstats.com/fight-details/ac0a1d87e9ff4085',
 'http://ufcstats.com/fight-details/78afed76d0e9a639',
 'http://ufcstats.com/fight-details/d40eae4425092219',
 'http://ufcstats.com/fight-details/ce0fda34122f3d48',
 'http://ufcstats.com/fight-details/4622ec182e7fed6b',
 'http://ufcstats.com/fight-details/c9283da5f4d7aaf4',
 'http://ufcstats.com/fight-details/4341ebb5a56e6943',
 'http://u

In [8]:
all_events_url = "http://ufcstats.com/statistics/events/completed?page=all"

In [9]:
browser.visit(all_events_url)
all_events_soup = bs(browser.html, "html.parser")
past_events = all_events_soup.find_all(class_="b-link b-link_style_black")
future_events = all_events_soup.find_all(class_="b-link b-link_style_white")

new_event_urls = []
future_event_urls = []
for event in future_events:
    future_url = event["href"]
    if future_url not in future_event_urls:
        future_event_urls.append(future_url)
        
for url in past_events:
    past_url = url["href"]
    if past_url not in event_urls:
        new_event_urls.append(past_url)
new_event_urls

[]

In [10]:
future_event_urls

['http://ufcstats.com/event-details/e57f0b06ea39e360']

In [11]:
future_events_data = []
future_fight_urls = []
for event in future_event_urls:
    browser.visit(event)
    event_soup = bs(browser.html, "html.parser")
    hrefs = event_soup.find_all(class_="b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click")
    
    
    event_date = browser.find_by_xpath("/html/body/section/div/div/div[1]/ul/li[1]").text
    event_name = browser.find_by_xpath("/html/body/section/div/h2/span").text
    event_location = browser.find_by_xpath("/html/body/section/div/div/div[1]/ul/li[2]").text
    
    event_info = {"event": event_name,
                 "date": event_date,
                 "location": event_location,
                 "event_url": event}
    
    future_events_data.append(event_info)
    
      
    for cols in hrefs:
        href = cols["data-link"]        
       
        if href not in fight_urls:
            future_fight_urls.append(href)
            
future_fight_urls

['http://ufcstats.com/fight-details/214b268e4a92f950',
 'http://ufcstats.com/fight-details/fad08eb7e0777593',
 'http://ufcstats.com/fight-details/4bb77b1d8efc53a9',
 'http://ufcstats.com/fight-details/1bde9cb65d31cc76',
 'http://ufcstats.com/fight-details/7c3ad06a520b8716',
 'http://ufcstats.com/fight-details/1be65a0d2c1383f6',
 'http://ufcstats.com/fight-details/4911094a4ee26868',
 'http://ufcstats.com/fight-details/a01e88062d4b2315',
 'http://ufcstats.com/fight-details/dff1dbb8fe1f31cb',
 'http://ufcstats.com/fight-details/43c391b251337c9b',
 'http://ufcstats.com/fight-details/cade0268dec56f3d',
 'http://ufcstats.com/fight-details/c13474b85d3ced32',
 'http://ufcstats.com/fight-details/40ce202eaa78860c']

In [12]:
future_event_df = pd.DataFrame(future_events_data)
future_event_df["date"] = future_event_df["date"].str.strip("DATE:")
future_event_df["location"] = future_event_df["location"].str.strip("LOCATION:")
future_event_df

,event,date,location,event_url
0,UFC Fight Night: Sandhagen vs. Dillashaw,"July 24, 2021","Las Vegas, Nevada, US",http://ufcstats.com/event-details/e57f0b06ea39...


In [13]:
future_event_df.to_csv("../data/future_event_data.csv")

In [14]:
new_events_data = []
new_fight_urls = []
for event in new_event_urls:
    browser.visit(event)
    event_soup = bs(browser.html, "html.parser")
    hrefs = event_soup.find_all(class_="b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click")
    
    
    event_date = browser.find_by_xpath("/html/body/section/div/div/div[1]/ul/li[1]").text
    event_name = browser.find_by_xpath("/html/body/section/div/h2/span").text
    event_location = browser.find_by_xpath("/html/body/section/div/div/div[1]/ul/li[2]").text
    
    event_info = {"event": event_name,
                 "date": event_date,
                 "location": event_location,
                 "event_url": event}
    
    new_events_data.append(event_info)
    
      
    for cols in hrefs:
        href = cols["data-link"]        
       
        if href not in fight_urls:
            new_fight_urls.append(href)
            
new_fight_urls

[]

In [15]:
new_event_df = pd.DataFrame(new_events_data)
new_event_df["date"] = new_event_df["date"].str.strip("DATE:")
new_event_df["location"] = new_event_df["location"].str.strip("LOCATION:")
new_event_df

KeyError: 'date'

In [16]:
event_frames = [new_event_df, event_df]
rolling_event_df = pd.concat(event_frames)
rolling_event_df = rolling_event_df.drop(columns="Unnamed: 0")
rolling_event_df

,event,date,location,event_url
0,UFC Fight Night: Makhachev vs. Moises,"July 17, 2021","Las Vegas, Nevada, US",http://ufcstats.com/event-details/40389d39a92f...
1,UFC 264: Poirier vs. McGregor 3,"July 10, 2021","Las Vegas, Nevada, US",http://ufcstats.com/event-details/8a0be41c0380...
2,UFC Fight Night: Gane vs. Volkov,"June 26, 2021","Las Vegas, Nevada, US",http://ufcstats.com/event-details/a1d6b308781a...
3,UFC Fight Night: Jung vs. Ige,"June 19, 2021","Las Vegas, Nevada, US",http://ufcstats.com/event-details/9ddfb3369a3b...
4,UFC 263: Adesanya vs. Vettori 2,"June 12, 2021","Glendale, Arizona, US",http://ufcstats.com/event-details/0b64d0fed453...
...,...,...,...,...
564,UFC 6: Clash of the Titans,"July 14, 1995","Casper, Wyoming, US",http://ufcstats.com/event-details/1c3f5e85b59e...
565,UFC 5: The Return of the Beast,"April 07, 1995","Charlotte, North Carolina, US",http://ufcstats.com/event-details/dedc3bb440d0...
566,UFC 4: Revenge of the Warriors,"December 16, 1994","Tulsa, Oklahoma, US",http://ufcstats.com/event-details/b60391da771d...
567,UFC 3: The American Dream,"September 09, 1994","Charlotte, North Carolina, US",http://ufcstats.com/event-details/1a49e0670dfa...


In [17]:
rolling_event_df.to_csv("../data/rolling_event_data.csv")

In [18]:
fighter_data = []
full_fight_data = []

for fight_url in new_fight_urls:
    
    browser.visit(fight_url)
    print(fight_url)
    
    event_name = browser.find_by_xpath("/html/body/section/div/h2/a").text
    winner_method = browser.find_by_xpath("/html/body/section/div/div/div[2]/div[2]/p[1]/i[1]/i[2]").text
    end_round = browser.find_by_xpath("/html/body/section/div/div/div[2]/div[2]/p[1]/i[2]").text
    end_time = browser.find_by_xpath("/html/body/section/div/div/div[2]/div[2]/p[1]/i[3]").text
    referee = browser.find_by_xpath("/html/body/section/div/div/div[2]/div[2]/p[1]/i[5]/span").text
    weight_class = browser.find_by_xpath("/html/body/section/div/div/div[2]/div[1]/i").text
    
    fighter_1_name = browser.find_by_xpath("/html/body/section/div/div/div[1]/div[1]/div/h3/a").text
    fighter_1_corner_color = "red"
    fighter_1_outcome = browser.find_by_xpath("/html/body/section/div/div/div[1]/div[1]/i").text
    fighter_1_KDs = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[2]/p[1]").text
    fighter_1_sig_str = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[3]/p[1]").text
    fighter_1_total_str = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[5]/p[1]").text
    fighter_1_TDs = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[6]/p[1]").text
    fighter_1_sub_atts = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[8]/p[1]").text
    fighter_1_reversals = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[9]/p[1]").text
    fighter_1_ctrl_time = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[10]/p[1]").text
    fighter_1_sig_head = browser.find_by_xpath("/html/body/section/div/div/table/tbody/tr/td[4]/p[1]").text
    fighter_1_sig_body = browser.find_by_xpath("/html/body/section/div/div/table/tbody/tr/td[5]/p[1]").text
    fighter_1_sig_leg = browser.find_by_xpath("/html/body/section/div/div/table/tbody/tr/td[6]/p[1]").text
    fighter_1_sig_distance = browser.find_by_xpath("/html/body/section/div/div/table/tbody/tr/td[7]/p[1]").text
    fighter_1_sig_clinch = browser.find_by_xpath("/html/body/section/div/div/table/tbody/tr/td[8]/p[1]").text
    fighter_1_sig_ground = browser.find_by_xpath("/html/body/section/div/div/table/tbody/tr/td[9]/p[1]").text
    
    fighter_2_name = browser.find_by_xpath("/html/body/section/div/div/div[1]/div[2]/div/h3/a").text
    fighter_2_corner_color = "blue"
    fighter_2_outcome = browser.find_by_xpath("/html/body/section/div/div/div[1]/div[2]/i").text
    fighter_2_KDs = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[2]/p[2]").text
    fighter_2_sig_str = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[3]/p[2]").text
    fighter_2_total_str = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[5]/p[2]").text
    fighter_2_TDs = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[6]/p[2]").text
    fighter_2_sub_atts = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[8]/p[2]").text
    fighter_2_reversals = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[9]/p[2]").text
    fighter_2_ctrl_time = browser.find_by_xpath("/html/body/section/div/div/section[2]/table/tbody/tr/td[10]/p[2]").text
    fighter_2_sig_head = browser.find_by_xpath("/html/body/section/div/div/table/tbody/tr/td[4]/p[2]").text
    fighter_2_sig_body = browser.find_by_xpath("/html/body/section/div/div/table/tbody/tr/td[5]/p[2]").text
    fighter_2_sig_leg = browser.find_by_xpath("/html/body/section/div/div/table/tbody/tr/td[6]/p[2]").text
    fighter_2_sig_distance = browser.find_by_xpath("/html/body/section/div/div/table/tbody/tr/td[7]/p[2]").text
    fighter_2_sig_clinch = browser.find_by_xpath("/html/body/section/div/div/table/tbody/tr/td[8]/p[2]").text
    fighter_2_sig_ground = browser.find_by_xpath("/html/body/section/div/div/table/tbody/tr/td[9]/p[2]").text

    
    fighter_1_info = {"name" : fighter_1_name,
                  "corner" : fighter_1_corner_color,
                  "weight_class": weight_class,
                  "opponent": fighter_2_name,
                  "event": event_name,
                 "outcome" : fighter_1_outcome,
                  "method" : winner_method,
                  "Round": end_round,
                  "Time": end_time,
                  "Significant_strikes": fighter_1_sig_str,
                  "Total_strikes": fighter_1_total_str,
                  "KDs": fighter_1_KDs,    
                  "TDs": fighter_1_TDs,
                  "Sub_Atts": fighter_1_sub_atts,
                  "Reversals": fighter_1_reversals,
                  "Control_time": fighter_1_ctrl_time,
                  "Sig_Str_head": fighter_1_sig_head,
                  "Sig_Str_head": fighter_1_sig_head,
                  "Sig_Str_body": fighter_1_sig_body,
                  "Sig_Str_leg": fighter_1_sig_leg,
                  "Sig_Str_distance": fighter_1_sig_distance,
                  "Sig_Str_clinch": fighter_1_sig_clinch,
                  "Sig_Str_ground": fighter_1_sig_ground,}
      
    fighter_2_info = {"name" : fighter_2_name,
                  "corner" : fighter_2_corner_color,
                  "weight_class": weight_class,
                  "opponent" : fighter_1_name,
                  "event" : event_name,
                 "outcome" : fighter_2_outcome,
                 "method" : winner_method,
                  "Round": end_round,
                  "Time": end_time,
                  "Significant_strikes": fighter_2_sig_str,
                  "Total_strikes": fighter_2_total_str,
                  "KDs": fighter_2_KDs,
                  "TDs": fighter_2_TDs,
                  "Sub_Atts": fighter_2_sub_atts,
                  "Reversals": fighter_2_reversals,
                  "Control_time": fighter_2_ctrl_time,
                  "Sig_Str_head": fighter_2_sig_head,
                  "Sig_Str_body": fighter_2_sig_body,
                  "Sig_Str_leg": fighter_2_sig_leg,
                  "Sig_Str_distance": fighter_2_sig_distance,
                  "Sig_Str_clinch": fighter_2_sig_clinch,
                  "Sig_Str_ground": fighter_2_sig_ground,}
    
    if fighter_1_outcome == "W":
        winner = "red"
    elif fighter_2_outcome == "W":
        winner = "blue"
    elif fighter_1_outcome == "NC":
        winner = "NC"
    elif fighter_1_outcome == "D":
        winner = "Draw"
    else:
        winner = "None"
        
    fight_info = {"red_name" : fighter_1_name,
                  "blue_name" : fighter_2_name,
                  "weight_class" : weight_class,
                  "event": event_name,
                  "winner" : winner,
                  "method" : winner_method,
                  "Round": end_round,
                  "Time": end_time,
                  "red_Significant_strikes": fighter_1_sig_str,
                  "red_Total_strikes": fighter_1_total_str,
                  "red_KDs": fighter_1_KDs,
                  "red_TDs": fighter_1_TDs,
                  "red_Sub_Atts": fighter_1_sub_atts,
                  "red_Reversals": fighter_1_reversals,
                  "red_Control_time": fighter_1_ctrl_time,
                  "red_Sig_Str_head": fighter_1_sig_head,
                  "red_Sig_Str_body": fighter_1_sig_body,
                  "red_Sig_Str_leg": fighter_1_sig_leg,
                  "red_Sig_Str_distance": fighter_1_sig_distance,
                  "red_Sig_Str_clinch": fighter_1_sig_clinch,
                  "red_Sig_Str_ground": fighter_1_sig_ground,
                  "blue_Significant_strikes": fighter_2_sig_str,
                  "blue_Total_strikes": fighter_2_total_str,
                  "blue_KDs": fighter_2_KDs,
                  "blue_TDs": fighter_2_TDs,
                  "blue_Sub_Atts": fighter_2_sub_atts,
                  "blue_Reversals": fighter_2_reversals,
                  "blue_Control_time": fighter_2_ctrl_time,
                  "blue_Sig_Str_head": fighter_2_sig_head,
                  "blue_Sig_Str_body": fighter_2_sig_body,
                  "blue_Sig_Str_leg": fighter_2_sig_leg,
                  "blue_Sig_Str_distance": fighter_2_sig_distance,
                  "blue_Sig_Str_clinch": fighter_2_sig_clinch,
                  "blue_Sig_Str_ground": fighter_2_sig_ground,
                  "url": fight_url}
        
    fighter_data.append(fighter_1_info)    
    fighter_data.append(fighter_2_info)
    full_fight_data.append(fight_info)

In [19]:
new_fights_df = pd.DataFrame(full_fight_data)
new_fights_df

""


In [23]:
all_fights_df = pd.read_csv("../data/rolling_fight_data.csv")
all_fights_df

,Unnamed: 0,red_name,blue_name,weight_class,event,winner,method,Round,Time,red_Significant_strikes,red_Total_strikes,red_KDs,red_TDs,red_Sub_Atts,red_Reversals,red_Control_time,red_Sig_Str_head,red_Sig_Str_body,red_Sig_Str_leg,red_Sig_Str_distance,red_Sig_Str_clinch,red_Sig_Str_ground,blue_Significant_strikes,blue_Total_strikes,blue_KDs,blue_TDs,blue_Sub_Atts,blue_Reversals,blue_Control_time,blue_Sig_Str_head,blue_Sig_Str_body,blue_Sig_Str_leg,blue_Sig_Str_distance,blue_Sig_Str_clinch,blue_Sig_Str_ground,url,Unnamed: 0.1,round
0,0,Islam Makhachev,Thiago Moises,LIGHTWEIGHT BOUT,UFC Fight Night: Makhachev vs. Moises,red,Submission,ROUND: 4,TIME: 2:38,61 of 92,148 of 194,0,3 of 6,1,1,8:11,28 of 56,31 of 34,2 of 2,22 of 41,27 of 31,12 of 20,13 of 44,18 of 51,0,1 of 2,1,0,3:24,6 of 30,5 of 11,2 of 3,9 of 39,3 of 3,1 of 2,http://ufcstats.com/fight-details/7d4c160dab6e...,NaN,NaN
1,1,Marion Reneau,Miesha Tate,WOMEN'S BANTAMWEIGHT BOUT,UFC Fight Night: Makhachev vs. Moises,blue,KO/TKO,ROUND: 3,TIME: 1:53,25 of 64,48 of 88,0,0 of 0,0,0,0:00,16 of 54,7 of 8,2 of 2,15 of 49,7 of 9,3 of 6,65 of 120,110 of 183,0,3 of 6,0,0,5:52,44 of 93,16 of 20,5 of 7,32 of 74,8 of 8,25 of 38,http://ufcstats.com/fight-details/f0b83b05f3dd...,NaN,NaN
2,2,Jeremy Stephens,Mateusz Gamrot,LIGHTWEIGHT BOUT,UFC Fight Night: Makhachev vs. Moises,blue,Submission,ROUND: 1,TIME: 1:05,0 of 0,2 of 2,0,0 of 0,0,0,0:03,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,1 of 1,1 of 1,0,1 of 2,1,0,0:46,1 of 1,0 of 0,0 of 0,1 of 1,0 of 0,0 of 0,http://ufcstats.com/fight-details/72105dd82d96...,NaN,NaN
3,3,Rodolfo Vieira,Dustin Stoltzfus,MIDDLEWEIGHT BOUT,UFC Fight Night: Makhachev vs. Moises,red,Submission,ROUND: 3,TIME: 1:54,49 of 88,52 of 93,0,3 of 4,1,0,2:10,42 of 81,2 of 2,5 of 5,49 of 88,0 of 0,0 of 0,67 of 192,83 of 209,0,0 of 3,0,1,0:09,53 of 170,11 of 18,3 of 4,64 of 187,2 of 3,1 of 2,http://ufcstats.com/fight-details/813444e10999...,NaN,NaN
4,4,Gabriel Benitez,Billy Quarantillo,FEATHERWEIGHT BOUT,UFC Fight Night: Makhachev vs. Moises,blue,KO/TKO,ROUND: 3,TIME: 3:40,57 of 110,67 of 122,1,0 of 0,1,1,0:50,38 of 89,13 of 15,6 of 6,44 of 89,6 of 9,7 of 12,100 of 156,179 of 259,1,3 of 7,1,0,8:40,73 of 126,20 of 23,7 of 7,61 of 107,11 of 12,28 of 37,http://ufcstats.com/fight-details/940701effe1d...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6171,6161,Orlando Wiet,Robert Lucarelli,OPEN WEIGHT BOUT,UFC 2: No Way Out,red,KO/TKO,NaN,TIME: 2:50,8 of 12,11 of 15,0,0 of 0,0,0,--,7 of 11,1 of 1,0 of 0,1 of 3,0 of 0,7 of 9,2 of 6,2 of 6,0,1 of 1,1,0,--,1 of 2,0 of 1,1 of 3,2 of 6,0 of 0,0 of 0,http://ufcstats.com/fight-details/3b020d4914b4...,6161.0,ROUND: 1
6172,6162,Frank Hamaker,Thaddeus Luster,OPEN WEIGHT BOUT,UFC 2: No Way Out,red,Submission,NaN,TIME: 4:52,2 of 3,14 of 15,0,1 of 1,3,1,--,2 of 3,0 of 0,0 of 0,1 of 1,0 of 0,1 of 2,0 of 0,0 of 0,0,0 of 1,0,0,--,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,http://ufcstats.com/fight-details/d917c8c7461b...,6162.0,ROUND: 1
6173,6163,Johnny Rhodes,David Levicki,OPEN WEIGHT BOUT,UFC 2: No Way Out,red,KO/TKO,NaN,TIME: 12:13,11 of 17,74 of 86,0,1 of 1,0,0,--,9 of 15,1 of 1,1 of 1,1 of 1,1 of 1,9 of 15,4 of 5,95 of 102,0,0 of 0,0,0,--,4 of 5,0 of 0,0 of 0,1 of 2,2 of 2,1 of 1,http://ufcstats.com/fight-details/ccee020be2e8...,6163.0,ROUND: 1
6174,6164,Patrick Smith,Ray Wizard,OPEN WEIGHT BOUT,UFC 2: No Way Out,red,Submission,NaN,TIME: 0:58,1 of 1,1 of 1,0,0 of 1,1,0,--,0 of 0,1 of 1,0 of 0,0 of 0,1 of 1,0 of 0,1 of 1,2 of 2,0,0 of 0,0,0,--,0 of 0,0 of 0,1 of 1,1 of 1,0 of 0,0 of 0,http://ufcstats.com/fight-details/4b9ae533ccb3...,6164.0,ROUND: 1


In [24]:
fight_frames = [new_fights_df, all_fights_df]
rolling_fights_df = pd.concat(fight_frames)
rolling_fights_df

,Unnamed: 0,red_name,blue_name,weight_class,event,winner,method,Round,Time,red_Significant_strikes,red_Total_strikes,red_KDs,red_TDs,red_Sub_Atts,red_Reversals,red_Control_time,red_Sig_Str_head,red_Sig_Str_body,red_Sig_Str_leg,red_Sig_Str_distance,red_Sig_Str_clinch,red_Sig_Str_ground,blue_Significant_strikes,blue_Total_strikes,blue_KDs,blue_TDs,blue_Sub_Atts,blue_Reversals,blue_Control_time,blue_Sig_Str_head,blue_Sig_Str_body,blue_Sig_Str_leg,blue_Sig_Str_distance,blue_Sig_Str_clinch,blue_Sig_Str_ground,url,Unnamed: 0.1,round
0,0,Islam Makhachev,Thiago Moises,LIGHTWEIGHT BOUT,UFC Fight Night: Makhachev vs. Moises,red,Submission,ROUND: 4,TIME: 2:38,61 of 92,148 of 194,0,3 of 6,1,1,8:11,28 of 56,31 of 34,2 of 2,22 of 41,27 of 31,12 of 20,13 of 44,18 of 51,0,1 of 2,1,0,3:24,6 of 30,5 of 11,2 of 3,9 of 39,3 of 3,1 of 2,http://ufcstats.com/fight-details/7d4c160dab6e...,NaN,NaN
1,1,Marion Reneau,Miesha Tate,WOMEN'S BANTAMWEIGHT BOUT,UFC Fight Night: Makhachev vs. Moises,blue,KO/TKO,ROUND: 3,TIME: 1:53,25 of 64,48 of 88,0,0 of 0,0,0,0:00,16 of 54,7 of 8,2 of 2,15 of 49,7 of 9,3 of 6,65 of 120,110 of 183,0,3 of 6,0,0,5:52,44 of 93,16 of 20,5 of 7,32 of 74,8 of 8,25 of 38,http://ufcstats.com/fight-details/f0b83b05f3dd...,NaN,NaN
2,2,Jeremy Stephens,Mateusz Gamrot,LIGHTWEIGHT BOUT,UFC Fight Night: Makhachev vs. Moises,blue,Submission,ROUND: 1,TIME: 1:05,0 of 0,2 of 2,0,0 of 0,0,0,0:03,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,1 of 1,1 of 1,0,1 of 2,1,0,0:46,1 of 1,0 of 0,0 of 0,1 of 1,0 of 0,0 of 0,http://ufcstats.com/fight-details/72105dd82d96...,NaN,NaN
3,3,Rodolfo Vieira,Dustin Stoltzfus,MIDDLEWEIGHT BOUT,UFC Fight Night: Makhachev vs. Moises,red,Submission,ROUND: 3,TIME: 1:54,49 of 88,52 of 93,0,3 of 4,1,0,2:10,42 of 81,2 of 2,5 of 5,49 of 88,0 of 0,0 of 0,67 of 192,83 of 209,0,0 of 3,0,1,0:09,53 of 170,11 of 18,3 of 4,64 of 187,2 of 3,1 of 2,http://ufcstats.com/fight-details/813444e10999...,NaN,NaN
4,4,Gabriel Benitez,Billy Quarantillo,FEATHERWEIGHT BOUT,UFC Fight Night: Makhachev vs. Moises,blue,KO/TKO,ROUND: 3,TIME: 3:40,57 of 110,67 of 122,1,0 of 0,1,1,0:50,38 of 89,13 of 15,6 of 6,44 of 89,6 of 9,7 of 12,100 of 156,179 of 259,1,3 of 7,1,0,8:40,73 of 126,20 of 23,7 of 7,61 of 107,11 of 12,28 of 37,http://ufcstats.com/fight-details/940701effe1d...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6171,6161,Orlando Wiet,Robert Lucarelli,OPEN WEIGHT BOUT,UFC 2: No Way Out,red,KO/TKO,NaN,TIME: 2:50,8 of 12,11 of 15,0,0 of 0,0,0,--,7 of 11,1 of 1,0 of 0,1 of 3,0 of 0,7 of 9,2 of 6,2 of 6,0,1 of 1,1,0,--,1 of 2,0 of 1,1 of 3,2 of 6,0 of 0,0 of 0,http://ufcstats.com/fight-details/3b020d4914b4...,6161.0,ROUND: 1
6172,6162,Frank Hamaker,Thaddeus Luster,OPEN WEIGHT BOUT,UFC 2: No Way Out,red,Submission,NaN,TIME: 4:52,2 of 3,14 of 15,0,1 of 1,3,1,--,2 of 3,0 of 0,0 of 0,1 of 1,0 of 0,1 of 2,0 of 0,0 of 0,0,0 of 1,0,0,--,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,http://ufcstats.com/fight-details/d917c8c7461b...,6162.0,ROUND: 1
6173,6163,Johnny Rhodes,David Levicki,OPEN WEIGHT BOUT,UFC 2: No Way Out,red,KO/TKO,NaN,TIME: 12:13,11 of 17,74 of 86,0,1 of 1,0,0,--,9 of 15,1 of 1,1 of 1,1 of 1,1 of 1,9 of 15,4 of 5,95 of 102,0,0 of 0,0,0,--,4 of 5,0 of 0,0 of 0,1 of 2,2 of 2,1 of 1,http://ufcstats.com/fight-details/ccee020be2e8...,6163.0,ROUND: 1
6174,6164,Patrick Smith,Ray Wizard,OPEN WEIGHT BOUT,UFC 2: No Way Out,red,Submission,NaN,TIME: 0:58,1 of 1,1 of 1,0,0 of 1,1,0,--,0 of 0,1 of 1,0 of 0,0 of 0,1 of 1,0 of 0,1 of 1,2 of 2,0,0 of 0,0,0,--,0 of 0,0 of 0,1 of 1,1 of 1,0 of 0,0 of 0,http://ufcstats.com/fight-details/4b9ae533ccb3...,6164.0,ROUND: 1


In [25]:
rolling_fights_df.to_csv("../data/rolling_fight_data.csv")

In [26]:
rolling_fight_urls = rolling_fights_df["url"]
rolling_fight_url_df = pd.DataFrame(rolling_fight_urls)
rolling_fight_url_df

,url
0,http://ufcstats.com/fight-details/7d4c160dab6e...
1,http://ufcstats.com/fight-details/f0b83b05f3dd...
2,http://ufcstats.com/fight-details/72105dd82d96...
3,http://ufcstats.com/fight-details/813444e10999...
4,http://ufcstats.com/fight-details/940701effe1d...
...,...
6171,http://ufcstats.com/fight-details/3b020d4914b4...
6172,http://ufcstats.com/fight-details/d917c8c7461b...
6173,http://ufcstats.com/fight-details/ccee020be2e8...
6174,http://ufcstats.com/fight-details/4b9ae533ccb3...


In [27]:
rolling_fight_url_df.to_csv("../data/rolling_fight_urls")

In [28]:
future_event_urls

['http://ufcstats.com/event-details/e57f0b06ea39e360']

In [29]:
future_fight_urls = []
future_event_data = []

for event in future_event_urls:
    browser.visit(event)
    event_soup = bs(browser.html, "html.parser")
    hrefs = event_soup.find_all(class_="b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click")
    
    
    event_date = browser.find_by_xpath("/html/body/section/div/div/div[1]/ul/li[1]").text
    event_name = browser.find_by_xpath("/html/body/section/div/h2/span").text
    event_location = browser.find_by_xpath("/html/body/section/div/div/div[1]/ul/li[2]").text
    
    event_info = {"event": event_name,
                 "date": event_date,
                 "location": event_location,
                 "event_url": event}
    
    future_event_data.append(event_info)
    
      
    for cols in hrefs:
        href = cols["data-link"]        
       
        if href not in future_fight_urls:
            future_fight_urls.append(href)
            
future_fight_urls

['http://ufcstats.com/fight-details/214b268e4a92f950',
 'http://ufcstats.com/fight-details/fad08eb7e0777593',
 'http://ufcstats.com/fight-details/4bb77b1d8efc53a9',
 'http://ufcstats.com/fight-details/1bde9cb65d31cc76',
 'http://ufcstats.com/fight-details/7c3ad06a520b8716',
 'http://ufcstats.com/fight-details/1be65a0d2c1383f6',
 'http://ufcstats.com/fight-details/4911094a4ee26868',
 'http://ufcstats.com/fight-details/a01e88062d4b2315',
 'http://ufcstats.com/fight-details/dff1dbb8fe1f31cb',
 'http://ufcstats.com/fight-details/43c391b251337c9b',
 'http://ufcstats.com/fight-details/cade0268dec56f3d',
 'http://ufcstats.com/fight-details/c13474b85d3ced32',
 'http://ufcstats.com/fight-details/40ce202eaa78860c']

In [30]:
future_fighter_data = []
future_fight_data = []

for fight_url in future_fight_urls:
    
    browser.visit(fight_url)
    print(fight_url)
    
    event_name = browser.find_by_xpath("/html/body/section/div/h2/a").text
    weight_class = browser.find_by_xpath("/html/body/section/div/div/div[2]/div[1]/i").text
    
    fighter_1_name = browser.find_by_xpath("/html/body/section/div/div/div[1]/div[1]/div/h3/a").text
    fighter_1_corner_color = "red"
    
    fighter_2_name = browser.find_by_xpath("/html/body/section/div/div/div[1]/div[2]/div/h3/a").text
    fighter_2_corner_color = "blue"
      
    fighter_1_info = {"name" : fighter_1_name,
                  "corner" : fighter_1_corner_color,
                  "weight_class": weight_class,
                  "opponent" : fighter_2_name,
                  "event" : event_name,
                 "outcome" : "tbd",
                 "method" : "tbd",
                  "Round": 0,
                  "Time": "0:00",
                  "Significant_strikes": "0 of 0",
                  "Total_strikes": "0 of 0",
                  "KDs": 0,
                  "TDs": "0 of 0",
                  "Sub_Atts": 0,
                  "Reversals": 0,
                  "Control_time": "0:00",
                  "Sig_Str_head": "0 of 0",
                  "Sig_Str_body": "0 of 0",
                  "Sig_Str_leg": "0 of 0",
                  "Sig_Str_distance": "0 of 0",
                  "Sig_Str_clinch": "0 of 0",
                  "Sig_Str_ground": "0 of 0",}
    
    fighter_2_info = {"name" : fighter_2_name,
                  "corner" : fighter_2_corner_color,
                  "weight_class": weight_class,
                  "opponent" : fighter_1_name,
                  "event" : event_name,
                 "outcome" : "tbd",
                 "method" : "tbd",
                  "Round": "0",
                  "Time": "0:00",
                  "Significant_strikes": "0 of 0",
                  "Total_strikes": "0 of 0",
                  "KDs": 0,
                  "TDs": "0 of 0",
                  "Sub_Atts": 0,
                  "Reversals": 0,
                  "Control_time": "0:00",
                  "Sig_Str_head": "0 of 0",
                  "Sig_Str_body": "0 of 0",
                  "Sig_Str_leg": "0 of 0",
                  "Sig_Str_distance": "0 of 0",
                  "Sig_Str_clinch": "0 of 0",
                  "Sig_Str_ground": "0 of 0",}
    
    fight_info = {"red_name" : fighter_1_name,
                  "blue_name" : fighter_2_name,
                  "weight_class" : weight_class,
                  "event": event_name,
                  "winner" : "tbd",
                  "method" : "tbd",
                  "Round": "0",
                  "Time": "0:00",
                  "red_Significant_strikes": "0 of 0",
                  "red_Total_strikes": "0 of 0",
                  "red_KDs": 0,
                  "red_TDs": "0 of 0",
                  "red_Sub_Atts": 0,
                  "red_Reversals": 0,
                  "red_Control_time": "0:00",
                  "red_Sig_Str_head": "0 of 0",
                  "red_Sig_Str_body": "0 of 0",
                  "red_Sig_Str_leg": "0 of 0",
                  "red_Sig_Str_distance": "0 of 0",
                  "red_Sig_Str_clinch": "0 of 0",
                  "red_Sig_Str_ground": "0 of 0",
                  "blue_Significant_strikes": "0 of 0",
                  "blue_Total_strikes": "0 of 0",
                  "blue_KDs": 0,
                  "blue_TDs": "0 of 0",
                  "blue_Sub_Atts": 0,
                  "blue_Reversals": 0,
                  "blue_Control_time": "0:00",
                  "blue_Sig_Str_head": "0 of 0",
                  "blue_Sig_Str_body": "0 of 0",
                  "blue_Sig_Str_leg": "0 of 0",
                  "blue_Sig_Str_distance": "0 of 0",
                  "blue_Sig_Str_clinch": "0 of 0",
                  "blue_Sig_Str_ground": "0 of 0",
                  "url": fight_url}
        
    future_fighter_data.append(fighter_1_info)    
    future_fighter_data.append(fighter_2_info)
    future_fight_data.append(fight_info)

http://ufcstats.com/fight-details/214b268e4a92f950
http://ufcstats.com/fight-details/fad08eb7e0777593
http://ufcstats.com/fight-details/4bb77b1d8efc53a9
http://ufcstats.com/fight-details/1bde9cb65d31cc76
http://ufcstats.com/fight-details/7c3ad06a520b8716
http://ufcstats.com/fight-details/1be65a0d2c1383f6
http://ufcstats.com/fight-details/4911094a4ee26868
http://ufcstats.com/fight-details/a01e88062d4b2315
http://ufcstats.com/fight-details/dff1dbb8fe1f31cb
http://ufcstats.com/fight-details/43c391b251337c9b
http://ufcstats.com/fight-details/cade0268dec56f3d
http://ufcstats.com/fight-details/c13474b85d3ced32
http://ufcstats.com/fight-details/40ce202eaa78860c


In [31]:
future_fight_df = pd.DataFrame(future_fight_data)
future_fight_df.to_csv("../data/future_fights_data.csv")
future_fight_df

,red_name,blue_name,weight_class,event,winner,method,Round,Time,red_Significant_strikes,red_Total_strikes,red_KDs,red_TDs,red_Sub_Atts,red_Reversals,red_Control_time,red_Sig_Str_head,red_Sig_Str_body,red_Sig_Str_leg,red_Sig_Str_distance,red_Sig_Str_clinch,red_Sig_Str_ground,blue_Significant_strikes,blue_Total_strikes,blue_KDs,blue_TDs,blue_Sub_Atts,blue_Reversals,blue_Control_time,blue_Sig_Str_head,blue_Sig_Str_body,blue_Sig_Str_leg,blue_Sig_Str_distance,blue_Sig_Str_clinch,blue_Sig_Str_ground,url
0,Cory Sandhagen,TJ Dillashaw,BANTAMWEIGHT BOUT,UFC Fight Night: Sandhagen vs. Dillashaw,tbd,tbd,0,0:00,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,http://ufcstats.com/fight-details/214b268e4a92...
1,Aspen Ladd,Macy Chiasson,WOMEN'S BANTAMWEIGHT BOUT,UFC Fight Night: Sandhagen vs. Dillashaw,tbd,tbd,0,0:00,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,http://ufcstats.com/fight-details/fad08eb7e077...
2,Kyler Phillips,Raulian Paiva,BANTAMWEIGHT BOUT,UFC Fight Night: Sandhagen vs. Dillashaw,tbd,tbd,0,0:00,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,http://ufcstats.com/fight-details/4bb77b1d8efc...
3,Darren Elkins,Darrick Minner,FEATHERWEIGHT BOUT,UFC Fight Night: Sandhagen vs. Dillashaw,tbd,tbd,0,0:00,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,http://ufcstats.com/fight-details/1bde9cb65d31...
4,Miranda Maverick,Maycee Barber,WOMEN'S FLYWEIGHT BOUT,UFC Fight Night: Sandhagen vs. Dillashaw,tbd,tbd,0,0:00,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,http://ufcstats.com/fight-details/7c3ad06a520b...
5,Mickey Gall,Jordan Williams,WELTERWEIGHT BOUT,UFC Fight Night: Sandhagen vs. Dillashaw,tbd,tbd,0,0:00,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,http://ufcstats.com/fight-details/1be65a0d2c13...
6,Punahele Soriano,Brendan Allen,MIDDLEWEIGHT BOUT,UFC Fight Night: Sandhagen vs. Dillashaw,tbd,tbd,0,0:00,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,http://ufcstats.com/fight-details/4911094a4ee2...
7,Shamil Abdurakhimov,Chris Daukaus,HEAVYWEIGHT BOUT,UFC Fight Night: Sandhagen vs. Dillashaw,tbd,tbd,0,0:00,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,http://ufcstats.com/fight-details/a01e88062d4b...
8,Nassourdine Imavov,Ian Heinisch,MIDDLEWEIGHT BOUT,UFC Fight Night: Sandhagen vs. Dillashaw,tbd,tbd,0,0:00,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,http://ufcstats.com/fight-details/dff1dbb8fe1f...
9,Adrian Yanez,Randy Costa,BANTAMWEIGHT BOUT,UFC Fight Night: Sandhagen vs. Dillashaw,tbd,tbd,0,0:00,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0,0 of 0,0,0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,http://ufcstats.com/fight-details/43c391b25133...
